# Kernel Ridge Regression with Tanimoto Kernel

**Hypothesis**: Kernel methods with chemical kernels may generalize better.
- Tanimoto kernel captures molecular similarity in a principled way
- Well-established approach in cheminformatics
- May have a different CV-LB relationship

**Baseline**: exp_030 CV 0.008298, LB 0.0877

In [1]:
# Standard imports
import numpy as np
import pandas as pd
import torch
from sklearn.kernel_ridge import KernelRidge
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import pairwise_kernels
import tqdm
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
torch.manual_seed(42)
torch.set_default_dtype(torch.double)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [2]:
# Data loading functions
DATA_PATH = '/home/data'

INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]
INPUT_LABELS_SINGLE_SOLVENT = ["Residence Time", "Temperature", "SOLVENT NAME"]
INPUT_LABELS_FULL_SOLVENT = ["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[INPUT_LABELS_FULL_SOLVENT]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[INPUT_LABELS_SINGLE_SOLVENT]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

print('Data loading functions defined')

Data loading functions defined


In [3]:
# Load feature lookups
SPANGE_DF = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)
DRFP_DF = pd.read_csv(f'{DATA_PATH}/drfps_catechol_lookup.csv', index_col=0)
ACS_PCA_DF = pd.read_csv(f'{DATA_PATH}/acs_pca_descriptors_lookup.csv', index_col=0)

# Filter DRFP to high-variance columns
drfp_variance = DRFP_DF.var()
nonzero_variance_cols = drfp_variance[drfp_variance > 0].index.tolist()
DRFP_FILTERED = DRFP_DF[nonzero_variance_cols]

print(f'Spange: {SPANGE_DF.shape}, DRFP filtered: {DRFP_FILTERED.shape}, ACS PCA: {ACS_PCA_DF.shape}')

Spange: (26, 13), DRFP filtered: (24, 122), ACS PCA: (24, 5)


In [4]:
# Tanimoto kernel for binary fingerprints
def tanimoto_kernel(X, Y=None):
    """Compute Tanimoto similarity kernel for binary fingerprints."""
    if Y is None:
        Y = X
    
    # Ensure binary
    X = (X > 0).astype(float)
    Y = (Y > 0).astype(float)
    
    # Compute intersection and union
    intersection = X @ Y.T
    X_sum = X.sum(axis=1, keepdims=True)
    Y_sum = Y.sum(axis=1, keepdims=True)
    union = X_sum + Y_sum.T - intersection
    
    # Avoid division by zero
    union = np.maximum(union, 1e-10)
    
    return intersection / union

print('Tanimoto kernel defined')

Tanimoto kernel defined


In [5]:
# Full Featurizer with DRFP for Tanimoto kernel
class FullFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.drfp_df = DRFP_FILTERED
        self.acs_pca_df = ACS_PCA_DF
        self.feats_dim = 2 + 3 + self.spange_df.shape[1] + self.drfp_df.shape[1] + self.acs_pca_df.shape[1]

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            A_drfp = self.drfp_df.loc[X["SOLVENT A NAME"]].values
            B_drfp = self.drfp_df.loc[X["SOLVENT B NAME"]].values
            A_acs = self.acs_pca_df.loc[X["SOLVENT A NAME"]].values
            B_acs = self.acs_pca_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                X_spange = B_spange * (1 - (1-pct)) + A_spange * (1-pct)
                X_drfp = B_drfp * (1 - (1-pct)) + A_drfp * (1-pct)
                X_acs = B_acs * (1 - (1-pct)) + A_acs * (1-pct)
            else:
                X_spange = A_spange * (1 - pct) + B_spange * pct
                X_drfp = A_drfp * (1 - pct) + B_drfp * pct
                X_acs = A_acs * (1 - pct) + B_acs * pct
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
            X_drfp = self.drfp_df.loc[X["SOLVENT NAME"]].values
            X_acs = self.acs_pca_df.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_kinetic, X_spange, X_drfp, X_acs])
    
    def get_drfp(self, X, flip=False):
        """Get just the DRFP fingerprints for Tanimoto kernel."""
        if self.mixed:
            A_drfp = self.drfp_df.loc[X["SOLVENT A NAME"]].values
            B_drfp = self.drfp_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                return B_drfp * (1 - (1-pct)) + A_drfp * (1-pct)
            else:
                return A_drfp * (1 - pct) + B_drfp * pct
        else:
            return self.drfp_df.loc[X["SOLVENT NAME"]].values

print(f'Full feature dimension: {FullFeaturizer().feats_dim}')

Full feature dimension: 145


In [6]:
# Kernel Ridge Regression Model with Tanimoto kernel
class KernelRidgeModel:
    def __init__(self, data='single', alpha=1.0):
        self.data_type = data
        self.featurizer = FullFeaturizer(mixed=(data=='full'))
        self.alpha = alpha
        self.models = []
        self.scaler = None
        self.X_train_drfp = None
        
    def train_model(self, X_train, y_train):
        # Get full features for scaling
        X_feat = self.featurizer.featurize(X_train)
        y_vals = y_train.values
        
        # Get DRFP for Tanimoto kernel
        X_drfp = self.featurizer.get_drfp(X_train)
        
        if self.data_type == 'full':
            X_flip = self.featurizer.featurize(X_train, flip=True)
            X_drfp_flip = self.featurizer.get_drfp(X_train, flip=True)
            X_all = np.vstack([X_feat, X_flip])
            X_drfp_all = np.vstack([X_drfp, X_drfp_flip])
            y_all = np.vstack([y_vals, y_vals])
        else:
            X_all, X_drfp_all, y_all = X_feat, X_drfp, y_vals
        
        self.scaler = StandardScaler()
        X_scaled = self.scaler.fit_transform(X_all)
        
        # Store training DRFP for kernel computation
        self.X_train_drfp = X_drfp_all
        self.X_train_scaled = X_scaled
        
        # Compute Tanimoto kernel on DRFP
        K_tanimoto = tanimoto_kernel(X_drfp_all)
        
        # Compute RBF kernel on scaled features
        gamma = 1.0 / X_scaled.shape[1]
        K_rbf = pairwise_kernels(X_scaled, metric='rbf', gamma=gamma)
        
        # Combine kernels (weighted average)
        K = 0.5 * K_tanimoto + 0.5 * K_rbf
        
        # Train separate model for each target
        self.models = []
        for i in range(3):
            # Kernel Ridge Regression: (K + alpha*I)^-1 * y
            n = K.shape[0]
            K_reg = K + self.alpha * np.eye(n)
            self.models.append(np.linalg.solve(K_reg, y_all[:, i]))
    
    def predict(self, X_test):
        X_feat = self.featurizer.featurize(X_test)
        X_drfp = self.featurizer.get_drfp(X_test)
        X_scaled = self.scaler.transform(X_feat)
        
        if self.data_type == 'full':
            X_flip = self.featurizer.featurize(X_test, flip=True)
            X_drfp_flip = self.featurizer.get_drfp(X_test, flip=True)
            X_flip_scaled = self.scaler.transform(X_flip)
        
        # Compute kernels
        K_tanimoto = tanimoto_kernel(X_drfp, self.X_train_drfp)
        gamma = 1.0 / X_scaled.shape[1]
        K_rbf = pairwise_kernels(X_scaled, self.X_train_scaled, metric='rbf', gamma=gamma)
        K = 0.5 * K_tanimoto + 0.5 * K_rbf
        
        preds = []
        for i, alpha in enumerate(self.models):
            pred = K @ alpha
            if self.data_type == 'full':
                K_tanimoto_flip = tanimoto_kernel(X_drfp_flip, self.X_train_drfp)
                K_rbf_flip = pairwise_kernels(X_flip_scaled, self.X_train_scaled, metric='rbf', gamma=gamma)
                K_flip = 0.5 * K_tanimoto_flip + 0.5 * K_rbf_flip
                pred_flip = K_flip @ alpha
                pred = (pred + pred_flip) / 2
            preds.append(pred)
        
        return torch.clamp(torch.tensor(np.column_stack(preds)), 0, 1)

print('KernelRidgeModel defined')

KernelRidgeModel defined


In [7]:
# Quick test on single fold
X_single, Y_single = load_data("single_solvent")
test_solvent = sorted(X_single["SOLVENT NAME"].unique())[0]
mask = X_single["SOLVENT NAME"] != test_solvent

model = KernelRidgeModel(data='single', alpha=0.1)
model.train_model(X_single[mask], Y_single[mask])
preds = model.predict(X_single[~mask])

actuals = Y_single[~mask].values
mse = np.mean((actuals - preds.numpy()) ** 2)
print(f'Test fold MSE: {mse:.6f}')
print(f'Predictions shape: {preds.shape}')

Test fold MSE: 0.037234
Predictions shape: torch.Size([37, 3])


In [8]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

import tqdm

X, Y = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = KernelRidgeModel(data='single', alpha=0.1)  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_single_solvent = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

0it [00:00, ?it/s]

1it [00:00,  6.37it/s]

2it [00:00,  7.77it/s]

3it [00:00,  7.59it/s]

4it [00:01,  1.98it/s]

...

20it [00:25,  1.79s/it]

21it [00:25,  1.31s/it]

22it [00:26,  1.19s/it]

23it [00:26,  1.14it/s]

24it [00:27,  1.13s/it]

In [9]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

X, Y = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = KernelRidgeModel(data='full', alpha=0.1)  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_full_data = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

0it [00:00, ?it/s]

1it [00:02,  2.84s/it]

2it [00:05,  2.75s/it]

3it [00:06,  2.12s/it]

4it [00:08,  1.80s/it]

...

10it [00:24,  3.08s/it]

11it [00:33,  4.86s/it]

12it [00:35,  4.18s/it]

13it [00:37,  3.38s/it]

13it [00:37,  2.86s/it]

In [10]:
########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("/home/submission/submission.csv", index=True)

########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [11]:
# Calculate CV score (for verification only - NOT part of submission)
X_single, Y_single = load_data("single_solvent")
X_full, Y_full = load_data("full")

# Get actuals in same order as predictions
actuals_single = []
for solvent in sorted(X_single["SOLVENT NAME"].unique()):
    mask = X_single["SOLVENT NAME"] == solvent
    actuals_single.append(Y_single[mask].values)
actuals_single = np.vstack(actuals_single)

actuals_full = []
ramps = X_full[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
for _, row in ramps.iterrows():
    mask = (X_full["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X_full["SOLVENT B NAME"] == row["SOLVENT B NAME"])
    actuals_full.append(Y_full[mask].values)
actuals_full = np.vstack(actuals_full)

# Get predictions
preds_single = submission_single_solvent[['target_1', 'target_2', 'target_3']].values
preds_full = submission_full_data[['target_1', 'target_2', 'target_3']].values

# Calculate MSE
mse_single = np.mean((actuals_single - preds_single) ** 2)
mse_full = np.mean((actuals_full - preds_full) ** 2)
n_single = len(actuals_single)
n_full = len(actuals_full)
overall_mse = (mse_single * n_single + mse_full * n_full) / (n_single + n_full)

print(f'\n=== CV SCORE VERIFICATION ===')
print(f'Single Solvent MSE: {mse_single:.6f} (n={n_single})')
print(f'Full Data MSE: {mse_full:.6f} (n={n_full})')
print(f'Overall MSE: {overall_mse:.6f}')
print(f'\nexp_030 baseline (GP+MLP+LGBM): CV 0.008298, LB 0.0877')
print(f'This (Kernel Ridge): CV {overall_mse:.6f}')

if overall_mse < 0.008298:
    improvement = (0.008298 - overall_mse) / 0.008298 * 100
    print(f'\n✓ IMPROVEMENT: {improvement:.2f}% better than exp_030!')
else:
    degradation = (overall_mse - 0.008298) / 0.008298 * 100
    print(f'\n✗ WORSE: {degradation:.2f}% worse than exp_030')
    print(f'\nNote: Even if CV is worse, LB might be better due to different CV-LB relationship.')


=== CV SCORE VERIFICATION ===
Single Solvent MSE: 0.017530 (n=656)
Full Data MSE: 0.017375 (n=1227)
Overall MSE: 0.017429

exp_030 baseline (GP+MLP+LGBM): CV 0.008298, LB 0.0877
This (Kernel Ridge): CV 0.017429

✗ WORSE: 110.03% worse than exp_030

Note: Even if CV is worse, LB might be better due to different CV-LB relationship.
